## 1)Importing the required libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from sklearn.utils import resample
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from sklearn.metrics import precision_recall_fscore_support, accuracy_score,confusion_matrix

## 2) Importing the dataset

In [2]:
n_iterations = 1000
training_data_gen = ImageDataGenerator()
traindata = training_data_gen.flow_from_directory(directory="/kaggle/input/surface-crack/train",target_size=(224,224))
validation_data_gen = ImageDataGenerator()
validdata = validation_data_gen.flow_from_directory(directory="/kaggle/input/surface-crack/valid", target_size=(224,224))

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


## 3)Defining the functions used

label(x) a function which is used to create the list yreal, which has true class values of testset images

pred(x) , a function which is used to determine class of the image .It takes in the output of last dense layer of our model and returns the class it belongs to.

test(L,path,images), given a list L and path under which the images are present, will append the predicted classes of those images to L

In [3]:
def label(x):
    if(x<100):
        #First 100 are crack-negative in test set so Yreal is 0
        return 0
    else:
        #Next 100 are crack-positive in test set so Yreal is 1
        return 1
def pred(x):
    if(x[0]>=x[1]):
        return 0
    else:
        return 1
def test(L,path):
    images=os.listdir(path)
    for imagnames in images:
        img_path = path+"/"+imagnames
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_batch)
        prediction = model.predict(img_preprocessed)
        L.append(pred(prediction[0]))

## 4)Defining the model

Model structure:

Inputimage(224 x 224 pixels rgb image)->VGG16->Dense(128 nodes)->Dense_output(2 nodes)

In [4]:
vgg = VGG16(weights='imagenet', include_top=True)
X= vgg.layers[-2].output
X = Dense(128, activation='relu')(X)
predictions = Dense(2, activation="softmax")(X)
model = Model(inputs = vgg.input, outputs = predictions)
model=Model(inputs=vgg.input, outputs = predictions)
checkpointname="vgg16_final.h5"
checkpoint = ModelCheckpoint(checkpointname, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

2023-01-19 15:47:46.874601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:47:46.875554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:47:46.885720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:47:46.886484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:47:46.887185: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

## 5)Training the model

We are training the model at a learning rate of 1e-4 over 100 epochs each iteration. we are training using training data , validating using validation data. The model gets saved after every iteration and predicts for the testset. when a testset accuracy of over 97% is achieved we stop and finalize our model.

In [5]:
yreal=[label(x) for x in range(0,200)]
negpath="/kaggle/input/surface-crack/test/Negative"
pospath="/kaggle/input/surface-crack/test/Positive"
for i in range(n_iterations):
    ypred=[]
    print("\n\nIteration number: ",i,"\n\n")
    model.compile(loss = "categorical_crossentropy", optimizer = SGD(lr=0.0001), metrics=["accuracy"])
    model.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= validdata, validation_steps=1, callbacks=[checkpoint,early])
    model.save_weights(checkpointname)
    test(ypred,negpath)
    test(ypred,pospath)
    precision, recall, f1_score, _ = precision_recall_fscore_support(yreal, ypred)
    avg_pre=(precision[0]+precision[1])/2
    acc = accuracy_score(yreal, ypred)
    print("\n Testing Accuracy is: ",acc)
    if(acc>0.97):
        cm=confusion_matrix(yreal,ypred)
        print("Model finalised.\nAccuracy: ",acc)
        print("Precision: ",precision)
        print("Recall: ",recall)
        print("F1: ",f1_score)
        print("Confusion matrix: ",cm)
        break

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '




Iteration number:  0 


Epoch 1/100


2023-01-19 15:47:50.822037: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-01-19 15:47:52.188200: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2/2 [==============================] - 15s 995ms/step - loss: 0.9508 - accuracy: 0.5781 - val_loss: 0.3857 - val_accuracy: 0.8438
Epoch 2/100
2/2 [==============================] - 1s 668ms/step - loss: 0.4757 - accuracy: 0.7812 - val_loss: 0.4819 - val_accuracy: 0.7812
Epoch 3/100
2/2 [==============================] - 1s 723ms/step - loss: 0.4828 - accuracy: 0.7344 - val_loss: 0.4788 - val_accuracy: 0.7500
Epoch 4/100
2/2 [==============================] - 1s 697ms/step - loss: 0.5596 - accuracy: 0.6875 - val_loss: 0.3475 - val_accuracy: 0.8438
Epoch 5/100
2/2 [==============================] - 1s 690ms/step - loss: 0.3853 - accuracy: 0.8750 - val_loss: 0.4680 - val_accuracy: 0.7500
Epoch 6/100
2/2 [==============================] - 1s 720ms/step - loss: 0.4153 - accuracy: 0.8281 - val_loss: 0.2655 - val_accuracy: 0.9688
Epoch 7/100
2/2 [==============================] - 1s 695ms/step - loss: 0.3968 - accuracy: 0.8281 - val_loss: 0.4354 - val_accuracy: 0.8125
Epoch 8/100
2/2 [=======

2023-01-19 15:50:28.692460: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.



 Testing Accuracy is:  0.975
Model finalised.
Accuracy:  0.975
Precision:  [1.         0.95238095]
Recall:  [0.95 1.  ]
F1:  [0.97435897 0.97560976]
Confusion matrix:  [[ 95   5]
 [  0 100]]


## 6)Predictions

In [6]:
predictpath="/kaggle/input/surface-crack/predict"
predict_images=os.listdir(predictpath)
predictions=[]
test(predictions,predictpath)
length=len(predictions)
for i in range(0,length):
    statement=""
    if(predictions[i]==1):
        statement=" Crack is present."
    else:
        statement=" No sign of cracks."
    print(predict_images[i]+statement)

IMG_1133.JPG Crack is present.
IMG_1129.JPG Crack is present.
IMG_1132.JPG Crack is present.
IMG_1130.JPG Crack is present.
IMG_1134.JPG Crack is present.
IMG_1131.JPG Crack is present.


## 7)Testing evaluation

In [8]:
ypred=[]
test(ypred,negpath)
test(ypred,pospath)
precision, recall, f1_score, _ = precision_recall_fscore_support(yreal, ypred)
avg_pre=(precision[0]+precision[1])/2
acc = accuracy_score(yreal, ypred)
cm=confusion_matrix(yreal,ypred)
print("Testing Accuracy is: ",acc)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1: ",f1_score)
print("Confusion matrix:\n",cm)

Testing Accuracy is:  0.975
Precision:  [1.         0.95238095]
Recall:  [0.95 1.  ]
F1:  [0.97435897 0.97560976]
Confusion matrix:
 [[ 95   5]
 [  0 100]]
